In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
sys.path.append('./waveglow/')

import librosa
import numpy as np
import os
import glob
import json

import torch
#from text import text_to_sequence, cmudict
#from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils

# load WaveGlow
waveglow_path = '../tacotron2/waveglow_256channels_universal_v5.pt' # or change to the latest version of the pretrained WaveGlow.
waveglow = torch.load(waveglow_path)['model']
waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.cuda().eval()
#from apex import amp
#waveglow, _ = amp.initialize(waveglow, [], opt_level="O3") # Try if you want to boost up synthesis speed.

/usr/local/lib/python3.8/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# If you are using your own trained model
model_dir = "./logs/myself/"
hps = utils.get_hparams_from_dir(model_dir)
checkpoint_path = utils.latest_checkpoint_path(model_dir)

# If you are using a provided pretrained model
# hps = utils.get_hparams_from_file("./configs/any_config_file.json")
# checkpoint_path = "/path/to/pretrained_model"
import json
with open(hps.data.dict) as f:
    symbols = json.load(f)
model = models.FlowGenerator(
    hps.data.symbols,
    out_channels=hps.data.n_mel_channels,
    **hps.model).to("cuda")

utils.load_checkpoint(checkpoint_path, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

#cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

# normalizing & type casting
def normalize_audio(x, max_wav_value=hps.data.max_wav_value):
    return np.clip((x / np.abs(x).max()) * max_wav_value, -32768, 32767).astype("int16")

./logs/myself/G_203.pth


/workspace/cpfs-data/glow-tts/modules.py:201: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:1940.)
  w_init = torch.qr(torch.FloatTensor(self.n_split, self.n_split).normal_())[0]


INFO:root:Loaded checkpoint './logs/myself/G_203.pth' (iteration 203)


In [7]:
tst_stn = ["a", "-", "o4", "|", "d", "-", "i4", "|", "l", "-", "i4", "|", "^", "z", "-", "o", "-", "ng2", "|", "t", "-", "o", "-", "ng3", "|", "*", "f", "-", "e", "-", "i1", "|", "sh", "-", "e4", "|", "er3", "|", "*", "h", "-", "e2", "|", "^", "f", "-", "u1", "|", "r", "-", "e", "-", "n5", "|", "&", "p", "-", "e", "-", "i2", "|", "t", "-", "o", "-", "ng2", "|", "^", "c", "-", "a", "-", "n1", "|", "g", "-", "u", "-", "a", "-", "n1", "|", "$"]
#if getattr(hps.data, "add_blank", False):
#    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'], cmu_dict)
#    text_norm = commons.intersperse(text_norm, len(symbols))
#else: # If not using "add_blank" option during training, adding spaces at the beginning and the end of utterance improves quality
#    tst_stn = " " + tst_stn.strip() + " "
#    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'], cmu_dict)
#sequence = np.array(text_norm)[None, :]
#print("".join([symbols[c] if c < len(symbols) else "<BNK>" for c in sequence[0]]))

sequence = np.array([symbols[x]+1 for x in tst_stn])
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long().unsqueeze(0)
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

In [12]:
with torch.no_grad():
  noise_scale = 1
  length_scale = 1.0
  sigma = 1
  (y_gen_tst, *_), *_, (attn_gen, *_) = model(x_tst, x_tst_lengths, gen=True, noise_scale=noise_scale, length_scale=length_scale)
  try:
    audio = waveglow.infer(y_gen_tst.half(), sigma=sigma)
  except:
    audio = waveglow.infer(y_gen_tst, sigma=sigma)
ipd.Audio(normalize_audio(audio[0].clamp(-1,1).data.cpu().float().numpy()), rate=hps.data.sampling_rate)